In [ ]:
#tax에 직접 병합하다 오류 발생 시 tax 전체가 손상될 위험이 있어 tax_merged를 생성 후 병합#

In [20]:
## 합칠 벡터 데이터베이스 불러오기##
#tax와 tax1 합치는 코드#

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

# 임베딩 모델 설정 (두 DB 모두 동일한 모델을 사용)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 첫 번째 데이터베이스 (메인 DB)
tax_db = Chroma(
    collection_name="tax_law",
    persist_directory="tax",
    embedding_function=embedding_model
)

# 두 번째 데이터베이스 (서브 DB)
tax1_db = Chroma(
    collection_name="tax1",
    persist_directory="tax1",
    embedding_function=embedding_model
)


In [21]:
# 메인 DB 문서 추출
tax_documents = tax_db.get()
tax_documents_converted = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(tax_documents['documents'], tax_documents['metadatas'])
]

# 서브 DB 문서 추출
tax1_documents = tax1_db.get()
tax1_documents_converted = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(tax1_documents['documents'], tax1_documents['metadatas'])
]


In [22]:
# 두 데이터베이스의 문서를 병합
merged_documents = tax_documents_converted + tax1_documents_converted


In [6]:
# 병합된 데이터베이스 생성
merged_db = Chroma.from_documents(
    documents=merged_documents,
    embedding=embedding_model,
    collection_name="tax_merged",
    persist_directory="tax_merged"
)


In [7]:
print(tax_db._embedding_function.model)
print(tax1_db._embedding_function.model)

text-embedding-3-large
text-embedding-3-large


In [8]:
# 병합된 데이터베이스 변경사항 저장
merged_db.persist()
print("✅ 병합된 데이터베이스가 저장되었습니다.")


✅ 병합된 데이터베이스가 저장되었습니다.


C:\Users\Playdata\AppData\Local\Temp\ipykernel_10760\2650310567.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  merged_db.persist()


In [9]:
##두 개의 벡터 데이터베이스 병합 후 테스트##
# 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 병합된 벡터 데이터베이스 불러오기
tax_merged_db = Chroma(
    collection_name="tax_merged",
    persist_directory="tax_merged",
    embedding_function=embedding_model
)


In [10]:
retriever = tax_merged_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10}
)

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.
        #추가한 부분#
        사용자의 질문이 정확한 법령명이나 조항을 다루고 있지 않더라도, 질문의 맥락과 키워드를 분석하여 가장 가까운 관련 법령 및 조항을 찾아 답변하세요.
        법령명에서 법과 같은 접미어가 생략된 경우에도 동일한 의미로 간주하세요.
        법령명과 조항 번호가 다소 부정확하게 입력되었더라도, AI가 가능한 한 사용자의 의도를 파악하여 올바른 법령과 조항으로 연결하세요.
        질문에 특정 조항(예: 제1조)이 포함된 경우, 해당 조항의 제목, 본문, 연혁, 주석을 종합적으로 확인해 답변하세요.
        
        {context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)

# 모델
model = ChatOpenAI(model="gpt-4o")

# Output Parser
parser = StrOutputParser()

# Chain 구성: retriever -> prompt_template -> model -> output_parser
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt_template | model | parser


In [13]:
# chain.invoke("사업소득이 4천만원 이하일때 소상공인 공제부금 소득공제의 한도를 알려줘")
# chain.invoke("소상공인 공제부금 소득공제의 한도는 얼마야? 사업소득이 1억을 넘어갔을 때")
chain.invoke("근로소득 산출세액이 130만원 이하일 때 세액공제금액을 알려줘")
# chain.invoke("어린이집 입소료는 교육비 세액공제 대상에 포함되어있어?")
# chain.invoke("근로제공 기간동안 지출한 비용에 대해서만 공제가능한 항목에 대해서 알려줘")



'근로소득 산출세액이 130만원 이하인 경우, 세액공제금액은 산출세액의 55%입니다. 이 규정은 근로소득세액공제를 통해 근로자의 세부담을 줄이기 위한 것으로, 소득이 낮은 경우 상대적으로 더 큰 혜택을 받을 수 있도록 설계되어 있습니다. \n\n추가적인 세부사항이나 사례가 필요하시다면 더 질문해 주세요.'

In [14]:
chain.invoke("연말정산시 소득·세액공제 증명서류 중점 확인사항")

'연말정산 시 소득·세액공제를 위한 증명서류를 제출할 때, 다음과 같은 사항을 중점적으로 확인해야 합니다:\n\n1. **주민등록표 등본**: 본인의 거주지와 세대주 여부 등을 확인할 수 있는 서류입니다. 소득공제 항목 중 일부는 세대주 여부에 따라 달라질 수 있습니다.\n\n2. **임대차계약서 사본**: 월세액 세액공제를 받기 위해 필요합니다. 임대차계약서에는 확정일자가 필요하지 않으므로 주의하세요.\n\n3. **현금영수증, 계좌이체 영수증, 무통장 입금증 등**: 월세액을 실제로 지급했음을 증명할 수 있는 서류입니다.\n\n4. **신용카드 사용금액 확인서**: 신용카드 등으로 지출한 금액에 대한 소득공제를 받을 수 있습니다. 연말정산 간소화 서비스에서 조회되지 않는 경우, 카드사로부터 확인서를 재발급받아야 합니다.\n\n5. **기타 문화비 지출 증빙 자료**: 도서·공연비 등에 대한 지출 증빙이 필요한 경우, 관련 영수증이나 확인서를 제출해야 합니다.\n\n6. **연금납입 확인서**: 연금계좌에 대한 납입액을 확인하는 서류로, 연금보험료 공제를 받기 위해 필요합니다.\n\n이 외에도 각자의 상황에 따라 추가적인 서류가 필요할 수 있으며, 제출하는 서류가 모두 실제 지출을 증명할 수 있는 정확한 자료인지 확인하는 것이 중요합니다. 필요한 서류를 빠짐없이 준비하여 원천징수의무자에게 제출해야 소득·세액공제를 제대로 받을 수 있습니다.'

In [16]:
chain.invoke("신용카드 소득공제율을 알려줘")

'2023년의 신용카드 소득공제율은 사용 시기와 지출 항목에 따라 다르게 적용됩니다. 다음은 주요 항목에 대한 공제율입니다:\n\n1. **일반 신용카드 사용금액**:\n   - 2023년 1월 1일부터 3월 31일까지 사용한 금액에 대해서는 30%의 공제율이 적용됩니다.\n   - 2023년 4월 1일부터 12월 31일까지 사용한 금액에 대해서는 40%의 공제율이 적용됩니다.\n\n2. **전통시장 사용금액**:\n   - 2023년 1월 1일부터 3월 31일까지 사용한 금액에 대해서는 40%의 공제율이 적용됩니다.\n   - 2023년 4월 1일부터 12월 31일까지 사용한 금액에 대해서는 50%의 공제율이 적용됩니다.\n\n3. **대중교통 사용금액**:\n   - 2023년 전체 기간 동안 대중교통 사용금액에 대해서는 80%의 공제율이 적용됩니다.\n\n이 외에도 특정 조건에 따라 다른 공제율이 적용될 수 있으므로, 자신의 상황에 맞는 정확한 정보는 국세청의 연말정산 간소화 서비스나 관련 법령을 통해 확인하는 것이 좋습니다.'

In [17]:
##tax_merged에 추가로 새로운 벡터DB 병합합할 때 사용하는 코드##
##preprocess_ks_주요공제계산사례.ipynb로 테스트##

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

# 1. 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 2. 기존 병합된 데이터베이스 불러오기
tax_merged_db = Chroma(
    collection_name="tax_merged",
    persist_directory="tax_merged",
    embedding_function=embedding_model
)

##이부분 바꿔주기##
# 3. 새로운 데이터베이스 불러오기
tax_new_db = Chroma(
    collection_name="tax_etc",
    persist_directory="vector_store/chroma/tax_etc_3",
    embedding_function=embedding_model
)

# 4. 새로운 DB 문서 추출 및 변환
tax_new_documents = tax_new_db.get()
tax_new_documents_converted = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(tax_new_documents['documents'], tax_new_documents['metadatas'])
]

# 5. 새로운 문서를 병합된 데이터베이스에 추가
tax_merged_db.add_documents(documents=tax_new_documents_converted)

# 6. 변경사항 저장
tax_merged_db.persist()
print("✅ 새로운 데이터베이스가 병합된 데이터베이스에 추가되었습니다.")


✅ 새로운 데이터베이스가 병합된 데이터베이스에 추가되었습니다.


In [29]:
# chain.invoke("신용카드 소득공제율을 알려줘")
chain.invoke("총급여 9천만원인 근로자의 신용카드 등 소득공제액을 알려줘")

'총급여 9천만원인 근로자의 신용카드 등 소득공제액 계산은 다음과 같습니다.\n\n1. **최저 사용 금액**: 총급여의 25%\n   - 9,000만원 × 25% = 2,250만원\n\n2. **사용 금액 합계**: 5,800만원\n   - 신용카드: 3,100만원 (대중교통 200만원 포함)\n   - 현금영수증: 1,800만원 (전통시장 350만원 포함)\n   - 체크카드: 900만원 (도서공연 270만원 포함)\n\n3. **소득공제 가능 여부**:\n   - 사용 금액 5,800만원이 최저 사용 금액 2,250만원을 초과하므로 소득공제 가능\n\n4. **소득공제 금액 계산**:\n   - **기본 공제 금액**: 1,212.5만원 (한도 250만원)\n     - 전통시장 사용분: 166만원\n     - 대중교통 사용분: 160만원\n     - 현금영수증 및 직불·선불카드 사용분: 789만원\n     - 신용카드 사용분: 435만원\n     - 최저 사용 금액에 해당하는 소득공제 금액: 337.5만원\n\n   - **추가 한도 적용에 따른 소득공제 금액**: 200만원\n     - Min{한도 초과액(1,212.5만원 - 300만원), (전통시장 및 대중교통 사용분 한도), 200만원}\n\n5. **총 소득공제 금액**: 450만원\n\n이렇게 계산된 신용카드 등 소득공제 금액은 총 450만원입니다. 이 계산에는 소득공제 추가분 150만원이 포함되어 있습니다.'

In [ ]:
# chain.invoke("근로자가 과세기간에 고향사랑기부금으로 20만원을 기부한 경우 기부금 세액공제금액은 얼마인가요?")
chain.invoke("근로자가 과세기간에 고향사랑기부금으로 8만원을 기부한 경우 기부금 세액공제금액은 얼마인가요?")

'근로자가 과세기간에 고향사랑기부금으로 8만원을 기부한 경우, 세액공제 금액을 계산하기 위해서는 조세특례제한법 제58조에 따른 세액공제율을 적용해야 합니다.\n\n고향사랑 기부금에 대한 세액공제는 다음과 같이 계산됩니다:\n- 10만원 이하의 금액에 대해서는 기부금의 100/110을 세액공제합니다.\n\n따라서, 8만원을 기부한 경우:\n- 8만원 × 100/110 = 72,727원\n\n따라서, 근로자의 고향사랑기부금 세액공제 금액은 72,727원입니다.'